In [1]:
########## import some useful package ##########

import time
t1 = time.time()

import uproot
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm

import ripser
import persim

In [2]:
########## read the data ##########

processes = ["hhvv"]
event_type = [str(i)+"_500K" for i in processes]

Xsection = [4.0360050e-02]     ### unit: fb
Luminosity = 1000   ### unit: fb^-1
simulation_num = 500000

features = ["VLCjetR10N2", "VLCjetR10N2.PT", "VLCjetR10N2.Mass", "VLCjetR10N2.Constituents", "EFlowTrack.fUniqueID", "EFlowTrack.PT", "EFlowTrack.Eta", "EFlowTrack.Phi", "EFlowTrack.PID", "EFlowPhoton.fUniqueID", "EFlowPhoton.ET", "EFlowPhoton.Eta", "EFlowPhoton.Phi", "EFlowNeutralHadron.fUniqueID", "EFlowNeutralHadron.ET", "EFlowNeutralHadron.Eta", "EFlowNeutralHadron.Phi"]

##### set data path #####

event_path = []
for type in event_type:
    event_path.append("/data/mucollider/two_boosted/" + type + "/delphes_output.root")

##### get the data file #####

data_file =[]
for path in event_path:
    data_file.append(uproot.open(path))
    
##### read data with features #####
events = []     ### total events
for process in processes:
    tmp_events = []
    for feature in features:
        tmp_events.append(data_file[processes.index(process)]["Delphes;1"][feature].array())
    tmp_events = np.expand_dims(tmp_events, axis=-1)
    tmp_events = tmp_events.transpose((1,0,2))
    tmp_events = np.squeeze(tmp_events,axis=(2,))
    events.append(tmp_events)
    print("Time:{:^8.4f}(s)".format(time.time()-t1))
del tmp_events

/usr/local/lib/python3.8/dist-packages/awkward/array/base.py:394: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return cls.numpy.array(value, copy=False)


Time:38.5297 (s)


In [3]:
########## define useful function ##########

##### calculate significance #####

def significance(s,b):
    return np.sqrt(2*((s+b)*np.log(1+s/b)-s))

##### count event number #####

def count(events):
    events_num = []
    for i, process in enumerate(processes):
        events_num.append(len(events[processes.index(process)]))
        print("There are", events_num[i], process, "events. Corresponding cross section:", Xsection[processes.index(process)]*events_num[i]/simulation_num, "(fb)")
        
##### select if Fat Jet >= 2 #####
        
def Fat_Jet_selection(events):
    where1 = np.where(events[:,features.index("VLCjetR10N2")]>=2)
    return events[where1]

##### select if M_jet_leading > XX GeV #####

def mass_leading_selection(events, low_mass_cut, high_mass_cut):
    where1 = []
    for i in range(len(events)):
        switch=1
        if events[i][features.index("VLCjetR10N2.Mass")][0]<low_mass_cut:
            switch=0
        if events[i][features.index("VLCjetR10N2.Mass")][0]>high_mass_cut:
            switch=0
        if switch==1:
            where1.append(i)
    return events[where1]

##### select if M_jet_subleading > XX GeV #####

def mass_subleading_selection(events, low_mass_cut, high_mass_cut):
    where1 = []
    for i in range(len(events)):
        switch=1
        if events[i][features.index("VLCjetR10N2.Mass")][1]<low_mass_cut:
            switch=0
        if events[i][features.index("VLCjetR10N2.Mass")][1]>high_mass_cut:
            switch=0
        if switch==1:
            where1.append(i)
    return events[where1]

##### select if pT_J_leading > XX GeV #####

def pT_leading_selection(events, low_pT_cut, high_pT_cut):
    where1 = []
    for i in range(len(events)):
        switch = 1
        if events[i][features.index("VLCjetR10N2.PT")][0]<low_pT_cut:
            switch=0
        if events[i][features.index("VLCjetR10N2.PT")][0]>high_pT_cut:
            switch=0
        if switch==1:
            where1.append(i)
    return events[where1]

##### select if pT_J_subleading > XX GeV #####

def pT_subleading_selection(events, low_pT_cut, high_pT_cut):
    where1 = []
    for i in range(len(events)):
        switch = 1
        if events[i][features.index("VLCjetR10N2.PT")][1]<low_pT_cut:
            switch=0
        if events[i][features.index("VLCjetR10N2.PT")][1]>high_pT_cut:
            switch=0
        if switch==1:
            where1.append(i)
    return events[where1]


In [6]:
event = events[processes.index("hhvv")][4]

In [100]:
##### set jet constituents ID #####

leadingJ_constituents = event[features.index("VLCjetR10N2.Constituents")][0][:-1]
subleadingJ_constituents= event[features.index("VLCjetR10N2.Constituents")][1][:-1]

##### construct infromation for all EFlow particles in this events and sort by PT #####

all_ID = np.concatenate((event[features.index("EFlowTrack.fUniqueID")], event[features.index("EFlowPhoton.fUniqueID")], event[features.index("EFlowNeutralHadron.fUniqueID")]))
all_PT = np.concatenate((event[features.index("EFlowTrack.PT")], event[features.index("EFlowPhoton.ET")], event[features.index("EFlowNeutralHadron.ET")]))
all_Eta = np.concatenate((event[features.index("EFlowTrack.Eta")], event[features.index("EFlowPhoton.Eta")], event[features.index("EFlowNeutralHadron.Eta")]))
all_Phi = np.concatenate((event[features.index("EFlowTrack.Phi")], event[features.index("EFlowPhoton.Phi")], event[features.index("EFlowNeutralHadron.Phi")]))
all_PID = np.concatenate((event[features.index("EFlowTrack.PID")], np.full(len(event[features.index("EFlowPhoton.fUniqueID")]), 22), np.full(len(event[features.index("EFlowNeutralHadron.fUniqueID")]), 0)))

sort_order = np.argsort(all_PT)[::-1]   ### from large PT to low PT

all_ID = all_ID[sort_order]
all_PT = all_PT[sort_order]
all_Eta = all_Eta[sort_order]
all_Phi = all_Phi[sort_order]
all_PID = all_PID[sort_order]

##### find Jet particle in EFlow particle #####

where_leading = np.where(np.isin(all_ID, leadingJ_constituents))[0][0:20]
where_subleading = np.where(np.isin(all_ID, subleadingJ_constituents))[0][0:20]

In [101]:
all_PT[where_leading]

array([20.046757 , 18.458828 , 17.51435  , 15.988174 , 14.039683 ,
       11.956813 , 10.794358 ,  9.736152 ,  8.739957 ,  8.207929 ,
        8.175008 ,  7.8207874,  7.3548427,  6.830789 ,  6.764571 ,
        6.6019106,  5.871273 ,  5.4466505,  5.2951612,  5.285693 ],
      dtype=float32)

In [102]:
all_PT[where_subleading]

array([35.125492 , 33.946125 , 31.306957 , 15.392311 , 14.333414 ,
       12.122381 , 10.708157 ,  9.3053255,  8.300904 ,  8.081643 ,
        7.8479457,  7.404683 ,  6.750331 ,  6.643255 ,  6.106781 ,
        4.912437 ,  4.6687684,  3.8101125,  2.8238428,  2.780826 ],
      dtype=float32)

In [124]:
len(events[processes.index("hhvv")][885][features.index("VLCjetR10N2.Constituents")][1])

18